### 2. ООП и лингвистические библиотеки

#### Задача 1. 

У вас есть файл с разборами conllu (можете взять любой, например, [тут](https://github.com/dialogue-evaluation/GramEval2020), или получить его с помощью UDPipe online). Нужно просмотреть все примеры предложений с тегом dislocated и тегом discourse: напишите скрипт, который будет читать файл, находить все такие предложения и печатать: 1) сам текст предложения 2) слово, имеющее искомый тег. Если тег не был найден в файле, нужно об этом сообщить. Постарайтесь оформить вывод таким образом, чтобы это было удобно читать. 

In [8]:
import pyconll

text = pyconll.load_from_file(r"C:\Users\lizao\Downloads\GramEval2020-SynTagRus-dev.conllu")

for sentence in text:
    for token in sentence:
        if token.deprel == 'discourse':
            print(f'{token.form, token.deprel}: {" ".join([token.form for token in sentence])}')


('то', 'discourse'): - Иметь - то имеют , - сказала Марья Поликарповна .
('Ну', 'discourse'): " Ну , надо же , - подумал Илья Ильич , - так просто сказала , ходя с восьмерок : " Мне , девки , недельки три еще - и до свидания " .


#### Задача 2 (5 баллов). 

Напишем программу, имитирующую помощника-лингвиста. Наш помощник - это экземпляр специального класса. К этому помощнику пользователь может обращаться за помощью (через диалог в консоли). Помощник должен уметь: переводить с одного языка на другой; сообщать часть речи для указанного слова в определенном языке; рисовать деревья зависимостей; пытаться дать определение многозначному слову по контексту (только для английского языка). Советую сперва написать более общий класс "бот-помощник", а от него наследовать класс "помощник-лингвист". Подумайте, как лучше структурировать код. Пользователь не обязан только выбирать из предложенного списка пунктов: попробуйте сделать так, чтобы с программой можно было осмысленно общаться, например:

    >>> Переведи, пожалуйста, на английский "сегодня хорошая погода".
    >>> Вот: It's a good weather today.
    >>> Переведи "Я хочу изучать программирование".
    >>> На какой язык? Я знаю такие языки: английский...
    
    >>> Подбери определение для слова book в предложении "I want to book a room in this hotel".
    >>> Book в этом предложении, вероятно, означает "arrange for and reserve (something for someone else) in advance".
    
    >>> Какая часть речи у слова "печь" в русском языке?
    >>> Может быть существительное, а может быть глагол.
    
С этим вам могут помочь регулярные выражения.

Доп. балл получите, если научите своего помощника предоставлять ссылку на страницу википедии по запросу ("найди мне страницу википедии про ..."). 

In [7]:
import googletrans, spacy, pymorphy2, re 
from spacy import displacy

class Bot:
    def __init__(self):
        self = self
    def chat(self, inp):
        vars = r'(пере(?:веди)|(?:вод))|((?:часть речи)|(?:POS))|(дерево (?:зависимости|ей)*)|(опредение|значение|смысл)'
        m = re.match(vars, inp, re.I) # почему inp не передается как строка
        if m.group(1):
            lang = re.match(r'\b\w+ий\b', inp)
            sent = re.match(r'\'|\".+\'|\"', inp)
            if lang.group():
                Linguist.translate(sent, lang)
            else:
                print('на какой язык перевести?')
                n = input()
                lang = re.match(r'\'|\".+\'|\"', n)
                Linguist.translate(sent, lang)
        elif m.group(2):
            word = re.match(r'\'|\".+\'|\"', inp)
            Linguist.part_speech(word)
        elif m.group(3):
            sent = re.match(r'\'|\".+\'|\"', inp)
            Linguist.parse_tree(sent)
        elif m.group(4):
            word = re.match(r'\'|\".+\'|\"', inp)
            Linguist.definition(word)
    def prosess(self):
        while True:
            question = input()
            self.chat(question)

class Linguist(Bot):
    def __init__(self):
        self = self
    def translate(self, text, lang2):
        trans = googletrans.Translator()
        return trans.translate(text, dest = lang2) # можно ли сюда передавать только dest?
    def part_speech(self, word):
        morph = pymorphy2.MorphAnalyzer()
        parse = morph.parse(word)
        return parse[0].tag.POS
    def parse_tree(self, sent):
        trans = googletrans.Translator()
        lang = trans.detect(sent)
        models = ["en_core_web_sm", "nl_core_news_sm", "fr_core_news_sm", "de_core_news_sm", "ru_core_news_sm"]
        if lang == 'en':
            nlp = spacy.load("en_core_web_sm")
        elif lang == 'ru':
            nlp = spacy.load("ru_core_news_sm")
        doc = nlp(sent)
        return displacy.serve(doc, style="dep")
    def definition(self, word):
        # а как какать
        pass

l = Bot()
l.prosess()
    

AttributeError: 'NoneType' object has no attribute 'group'

In [9]:
import re
print(re.match(r'\b\w+ий\b', "переведи на английский"))

None


#### Задача 3 (2 балла). 

Сымитируйте с помощью классов работу ресторана. У нас должен быть общий класс "ресторан", у которого есть официанты и повара; также есть класс "еда", который умеют готовить повара, и класс "посетитель", который приходит в ресторан и заказывает у официанта еду. Посетитель с определенным количеством денег приходит в ресторан, официант предлагает ему меню, посетитель выбирает, какое блюдо он хочет заказать. Если у него не хватает денег, он не может это сделать! Официант должен передать заказ (если денег хватило) повару, а повар возвращает экземпляр класса "еда" с нужными атрибутами. Официант выдает заказ посетителю. Можете сделать возможность выбирать блюдо пользователю, а можно все рандомизировать, чтобы программа работала в режиме "один день из жизни ресторана": в цикле приходят один за другим посетители и что-то случайным образом заказывают, а программа только печатает логи в консоль.

In [12]:
import random

class Restoraunt:
    def __init__(self):
        self.menu = {"carbonara" : 400, "pizza" : 1200, "sushi" : 2000, "pizza_set" : 4000, "sushi_set" : 5000, "desert" : 500, "water" : 200, "beer" : 300}
    def work(self):
        while True:
            g = Guest(random.randint(200, 10000))
            Waiter.bring_menu(g) # как так нет гостя?

class Waiter(Restoraunt):
    def __init__(self, menu):
        super().__init__(menu)
    def bring_menu(self, guest):
        return guest.make_order(self.menu)
    def order_to_kitchen(self, order):
        return Chef.return_food(order)
    def give_order(self, food):
        print(f'the order{food} is ready')

class Chef:
    def __init__(self):
        self = self
    def return_food(self, order):
        return Waiter.give_order(order)

class Guest:
    def __init__(self, money):
        self.money = money
    def make_order(self, menu):
        able = {k : v for k, v in menu if v <= self.money}
        return Waiter.order_to_kitchen(random.choice([k for k in able])) 
    
r = Restoraunt()
r.work()



TypeError: Waiter.bring_menu() missing 1 required positional argument: 'guest'